In [1]:
from pathlib import Path

import numba as nb
import numpy as np
import scipy.stats as stats

from natsort import natsorted

import matplotlib.pyplot as plt


#plt.style.use("mike")
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')


import astropy.units as u

import cmasher as cmr

green = "#33FF86"
purple = "#CE33FF"

%matplotlib widget
from cosmogrb.universe.survey import Survey

import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from zusammen.stan_models.stan_model import get_model
from zusammen import AnalysisBuilder, DataSet
from zusammen.spectral_plot import display_posterior_model_counts

from threeML import update_logging_level

import arviz as av


update_logging_level("FATAL")


from astromodels import Band_Calderone, PointSource, Model


from threeML import JointLikelihood, DataList, display_spectrum_model_counts

from astromodels import Cutoff_powerlaw

import popsynth as ps

10:43:02 WARNING   The naima package is not available. Models that depend on it will not be         ]8;id=552207;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=456263;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py#51\51]8;;\
                  available                                                                                        

         WARNING   The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it  ]8;id=441984;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=92251;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py#72\72]8;;\
                  will not be available.                                                                           

10:43:03 WARNING   The ebltable package is not available. Models that depend on it will not be     ]8;id=474284;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/astromodels/functions/functions_1D/absorption.py\absorption.py]8;;\:]8;id=823320;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/astromodels/functions/functions_1D/absorption.py#37\37]8;;\
                  available                                                                                        

10:43:03 INFO      Starting 3ML!                                                                     ]8;id=157163;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=915946;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py#31\31]8;;\

         WARNING   no display variable set. using backend for graphics without display (agg)         ]8;id=835409;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=439900;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py#36\36]8;;\

...

         WARNING   Could not import plugin FermiLATLike.py. Do you have the relative instrument     ]8;id=853398;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=103342;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py#130\130]8;;\
                  software installed and configured?                                                               

         WARNING   No fermitools installed                                              ]8;id=340987;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py\lat_transient_builder.py]8;;\:]8;id=605124;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py#44\44]8;;\

         WARNING   Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=23851;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=586992;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py#343\343]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=702230;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=545217;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py#343\343]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal     ]8;id=189978;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=337105;file:///opt/homebrew/Caskroom/miniconda/base/envs/bachelor/lib/python3.7/site-packages/threeML/__init__.py#343\343]8;;\
                  performances in 3ML                                                                              

Import the survey and process the GRBs

In [2]:
survey = Survey.from_file('data/survey.h5')
ab = AnalysisBuilder(survey, use_bb=True)

AttributeError: 'Logger' object has no attribute 'warnings'

In [ ]:
ab.write_yaml("test_proc.yml")

In [3]:
ds = DataSet.from_yaml("test_proc.yml")

IndexError: list index out of range

In [ ]:
ds.to_hdf5_file("sgrb.h5")

In [ ]:
ds.to_hdf5_file("sgrb.h5", sig_threshold=40)

In [ ]:
ds = DataSet.from_hdf5_file('sgrb.h5')

In [ ]:
ds.to_stan_dict()

Make Stan model

In [ ]:
m = get_model("cpl_simple_chunked_gc")

In [ ]:
m.clean_model()

In [ ]:
m.build_model(opt_exp=True)

In [ ]:
data = ds.to_stan_dict()

n_threads = 2
n_chains = 2

fit = m.model.sample(
    data=data,
    parallel_chains=n_chains,
    chains=n_chains,
    # inits= {'alpha':-1.},
    threads_per_chain=n_threads,
    seed=1234,
    iter_warmup=1000,
    iter_sampling=500,
    max_treedepth=12,
    show_progress=True,
)

In [ ]:
m.model.sample?

Import Stan results into arviz

In [ ]:
res = av.from_cmdstanpy(fit)

In [ ]:
res.to_netcdf("inference_data/test_inference_data_energy_flux_4.nc")

In [ ]:
res = av.from_netcdf("inference_data/test_inference_data_exact_2.nc")

In [ ]:
res.sample_stats.tree_depth.max()

In [ ]:
%matplotlib widget
av.plot_trace(res)

In [ ]:
bc = Band_Calderone(opt=0, a =10, b=1e4)
bc.beta.fix = True

Load parameters  
    $\alpha$: parameter of CPL  
    $E_p = \log E_c$  
    $K$: normalization

In [ ]:
id = 0

In [ ]:
alpha = res.posterior.alpha.stack(sample=("chain", "draw")).values[id]

log_ec = res.posterior.log_ec.stack(sample=("chain", "draw")).values[id]

K  = res.posterior.K.stack(sample=("chain", "draw")).values[id]

epeak = res.posterior.epeak.stack(sample=("chain", "draw")).values[id]

energy_flux = res.posterior.energy_flux.stack(sample=("chain", "draw")).values[id]

gamma = res.posterior.gamma.stack(sample=("chain", "draw")).values[id]

log_Nrest = res.posterior.log_Nrest.stack(sample=("chain", "draw")).values[id]


div = res.sample_stats.diverging.stack(sample=("chain", "draw")).values

samples = np.vstack((K, alpha, 10.**log_ec))


dl = ds.get_data_list_of_interval(id)

In [ ]:
alpha

In [ ]:
div.sum()

In [ ]:
bc = Cutoff_powerlaw(piv=100)

bc.index.bounds = (None, None)
bc.K.bounds = (None, None)
bc.xc.bounds = (None, None)

model = Model(PointSource("ps",0,0, spectral_shape=bc))

In [ ]:
fig, ax = plt.subplots()


ax.scatter(ep, alpha, alpha=0.1)


In [ ]:
fig, ax = plt.subplots()


ax.scatter(K, alpha, alpha=0.1)

In [ ]:
display_posterior_model_counts(
    dl[1], model, samples.T[::20], min_rate=1e-99, shade=False
)

In [ ]:
cpl = Cutoff_powerlaw(piv=100,K=1e-1,xc=200)


dl = ds.get_data_list_of_interval(2)

model = Model(PointSource("ps",0,0, spectral_shape=cpl))

ba = JointLikelihood(model,DataList(*dl))

In [ ]:
ba.fit()

In [ ]:
display_spectrum_model_counts(ba)

In [ ]:
dl[0].view_count_spectrum()

In [ ]:
cpl